## Overview of MLB Homeruns Dataset

In [ ]:
import pandas as pd
mlb_homeruns_df = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')

In [ ]:
mlb_homeruns_df.head()

# Data Preprocessing & Training in Batches

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pre-trained layers
feature_extractor = Model(inputs=base_model.input, outputs=Flatten()(base_model.output))  # Extracted features

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(50, 224, 224, 3)),  # Apply CNN to each frame
    LSTM(256, return_sequences=False),  # LSTM processes extracted features
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  # Predict ExitVelocity, HitDistance, LaunchAngle
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=50):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.linspace(0, total_frames - 1, frame_count, dtype=int)  # Sample evenly
    
    for fid in frame_ids:
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
        frames.append(frame)
    
    cap.release()
    os.remove(video_path)  # Remove temp file
    return np.array(frames) if len(frames) == frame_count else None  # Ensure 10 frames

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

# Prepare training data
batch_size = 50
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

# Create temp directory for videos
os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []
    
    # Process videos with proper tqdm progress tracking
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])
    
    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X)  # Shape: (batch_size, 10, 224, 224, 3)
    y = np.array(y)  # Shape: (batch_size, 3)

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')
    
    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    # Clear the TensorFlow session to release memory
    K.clear_session()
    
    # Trigger garbage collection to free up unused memory
    gc.collect()

    # Clear variables (X and y data) that are no longer needed
    X = []
    y = []


## Making Prediction by Loading Model

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import os
import requests

# Load the trained model
model_path = "/kaggle/working/model_batch_8.h5"  # Replace 'X' with the latest batch number
model = tf.keras.models.load_model(model_path, compile=False)

# Recompile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',  # Explicitly set the loss function again
              metrics=['mae'])

# Function to preprocess new video
def preprocess_video(video_url, save_path, frame_count=50):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)

    cap = cv2.VideoCapture(save_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.linspace(0, total_frames - 1, frame_count, dtype=int)

    for fid in frame_ids:
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
        frames.append(frame)

    cap.release()
    os.remove(save_path)  # Remove temporary video file

    if len(frames) == frame_count:
        return np.array(frames)
    else:
        return None  # Return None if video does not have enough frames

# Path to the new video for prediction
row_index = 700
video_url = mlb_homeruns_df['video'][row_index]  # Replace with actual video URL
video_save_path = "temp_video.mp4"

# Preprocess video
video_frames = preprocess_video(video_url, video_save_path)

if video_frames is not None:
    video_frames = np.expand_dims(video_frames, axis=0)  # Add batch dimension (1, 50, 224, 224, 3)

    # Make prediction
    predictions = model.predict(video_frames)

    # Display predicted values
    print(f"Predicted ExitVelocity: {predictions[0][0]:.2f}")
    print(f"Predicted HitDistance: {predictions[0][1]:.2f}")
    print(f"Predicted LaunchAngle: {predictions[0][2]:.2f}")

    print(f"\n\nActual ExitVelocity: {mlb_homeruns_df['ExitVelocity'][row_index]}")
    print(f"Actual HitDistance: {mlb_homeruns_df['HitDistance'][row_index]}")
    print(f"Actual LaunchAngle: {mlb_homeruns_df['LaunchAngle'][row_index]}")
else:
    print("Error: Not enough valid frames in the video.")


In [ ]:
mlb_homeruns_df.head()

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pre-trained layers
feature_extractor = Model(inputs=base_model.input, outputs=Flatten()(base_model.output))  # Extracted features

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(60, 224, 224, 3)),  # Apply CNN to each frame
    LSTM(256, return_sequences=False),  # LSTM processes extracted features
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  # Predict ExitVelocity, HitDistance, LaunchAngle
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=60):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_ids = np.linspace(0, total_frames - 1, frame_count, dtype=int)  # Sample evenly
    
    for fid in frame_ids:
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if not ret:
            continue
        frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
        frames.append(frame)
    
    cap.release()
    os.remove(video_path)  # Remove temp file
    return np.array(frames) if len(frames) == frame_count else None 

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

# Prepare training data
batch_size = 30
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

# Create temp directory for videos
os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []
    
    # Process videos with proper tqdm progress tracking
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])
    
    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X)  # Shape: (batch_size, 10, 224, 224, 3)
    y = np.array(y)  # Shape: (batch_size, 3)

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')
    
    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    # Clear the TensorFlow session to release memory
    K.clear_session()
    
    # Trigger garbage collection to free up unused memory
    gc.collect()

    # Clear variables (X and y data) that are no longer needed
    X = []
    y = []


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze pre-trained layers
feature_extractor = Model(inputs=base_model.input, outputs=Flatten()(base_model.output))  # Extracted features

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(5, 224, 224, 3)),  # Apply CNN to each frame
    LSTM(256, return_sequences=False),  # LSTM processes extracted features
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  # Predict ExitVelocity, HitDistance, LaunchAngle
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to detect ball-and-bat collision
def detect_collision(cap):
    """
    Detects the frame where ball-and-bat collision occurs.
    This is a placeholder implementation; you may need to use object detection or motion analysis.
    """
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Start from the beginning
    prev_frame = None
    collision_frame_id = 0

    for frame_id in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for motion detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

        if prev_frame is None:
            prev_frame = gray_frame
            continue

        # Compute absolute difference between current and previous frame
        frame_diff = cv2.absdiff(prev_frame, gray_frame)
        _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # If significant motion is detected, assume collision
        if any(cv2.contourArea(contour) > 500 for contour in contours):  # Adjust threshold as needed
            collision_frame_id = frame_id
            break

        prev_frame = gray_frame

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to the beginning
    return collision_frame_id

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=5):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)

    # Detect ball-and-bat collision
    collision_frame_id = detect_collision(cap)
    print(f"Detected collision at frame {collision_frame_id}")

    # Extract frames starting from the collision frame
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, collision_frame_id)  # Start from collision frame

    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (224, 224)) / 255.0  # Normalize
        frames.append(frame)

    cap.release()
    os.remove(video_path)  # Remove temp file
    return np.array(frames) if len(frames) == frame_count else None

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

# Prepare training data
batch_size = 30
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

# Create temp directory for videos
os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []

    # Process videos with proper tqdm progress tracking
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])

    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X)  # Shape: (batch_size, 15, 224, 224, 3)
    y = np.array(y)  # Shape: (batch_size, 3)

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')

    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    # Clear the TensorFlow session to release memory
    K.clear_session()

    # Trigger garbage collection to free up unused memory
    gc.collect()

    # Clear variables (X and y data) that are no longer needed
    X = []
    y = []

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed, Input
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
def create_grayscale_resnet():
    # Use a custom input layer for grayscale images (1 channel)
    input_tensor = Input(shape=(224, 224, 1))
    
    # Replicate the single channel to match ResNet50's expected input (3 channels)
    replicated_input = tf.keras.layers.Concatenate()([input_tensor] * 3)
    
    # Load ResNet50 with replicated input
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=replicated_input)
    base_model.trainable = False  # Freeze pre-trained layers
    
    # Add a feature extraction layer
    feature_extractor = Model(inputs=input_tensor, outputs=Flatten()(base_model.output))
    return feature_extractor

feature_extractor = create_grayscale_resnet()

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(5, 224, 224, 1)),  # Grayscale input
    LSTM(256, return_sequences=False),  # LSTM processes extracted features
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  # Predict ExitVelocity, HitDistance, LaunchAngle
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to detect ball-and-bat collision
def detect_collision(cap):
    """
    Detects the frame where ball-and-bat collision occurs.
    This is a placeholder implementation; you may need to use object detection or motion analysis.
    """
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Start from the beginning
    prev_frame = None
    collision_frame_id = 0

    for frame_id in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for motion detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

        if prev_frame is None:
            prev_frame = gray_frame
            continue

        # Compute absolute difference between current and previous frame
        frame_diff = cv2.absdiff(prev_frame, gray_frame)
        _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # If significant motion is detected, assume collision
        if any(cv2.contourArea(contour) > 500 for contour in contours):  # Adjust threshold as needed
            collision_frame_id = frame_id
            break

        prev_frame = gray_frame

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to the beginning
    return collision_frame_id

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=5):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)

    # Detect ball-and-bat collision
    collision_frame_id = detect_collision(cap)
    # print(f"Detected collision at frame {collision_frame_id}")

    # Extract frames starting from the collision frame
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, collision_frame_id)  # Start from collision frame

    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Resize frame to 224x224
        frame = cv2.resize(frame, (224, 224))

        # Quantize pixel values to integers (0-255)
        frame = np.uint8(frame)

        # Keep the frame as grayscale (1 channel)
        frame = np.expand_dims(frame, axis=-1)  # Shape: (224, 224, 1)

        frames.append(frame)

    cap.release()
    os.remove(video_path)  # Remove temp file
    return np.array(frames) if len(frames) == frame_count else None

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

# Prepare training data
batch_size = 30
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

# Create temp directory for videos
os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []

    # Process videos with proper tqdm progress tracking
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])

    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X)  # Shape: (batch_size, 5, 224, 224, 1)
    y = np.array(y)  # Shape: (batch_size, 3)

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')

    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    # Clear the TensorFlow session to release memory
    K.clear_session()

    # Trigger garbage collection to free up unused memory
    gc.collect()

    # Clear variables (X and y data) that are no longer needed
    X = []
    y = []

MobileNet CNN & LSTM

In [2]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed, Input
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
def create_grayscale_mobilenet():
    # Use a custom input layer for grayscale images (1 channel)
    input_tensor = Input(shape=(224, 224, 1))
    
    # Replicate the single channel to match MobileNet's expected input (3 channels)
    replicated_input = tf.keras.layers.Concatenate()([input_tensor] * 3)
    
    # Load MobileNetV2 with replicated input
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=replicated_input, pooling='avg')
    base_model.trainable = False  # Freeze pre-trained layers
    
    # Add a feature extraction layer
    feature_extractor = Model(inputs=input_tensor, outputs=base_model.output)
    return feature_extractor

feature_extractor = create_grayscale_mobilenet()

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(5, 224, 224, 1)),  # Grayscale input
    LSTM(256, return_sequences=False),  # LSTM processes extracted features
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  # Predict ExitVelocity, HitDistance, LaunchAngle
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to detect ball-and-bat collision
def detect_collision(cap):
    """
    Detects the frame where ball-and-bat collision occurs.
    This is a placeholder implementation; you may need to use object detection or motion analysis.
    """
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Start from the beginning
    prev_frame = None
    collision_frame_id = 0

    for frame_id in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for motion detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

        if prev_frame is None:
            prev_frame = gray_frame
            continue

        # Compute absolute difference between current and previous frame
        frame_diff = cv2.absdiff(prev_frame, gray_frame)
        _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # If significant motion is detected, assume collision
        if any(cv2.contourArea(contour) > 500 for contour in contours):  # Adjust threshold as needed
            collision_frame_id = frame_id
            break

        prev_frame = gray_frame

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Reset to the beginning
    return collision_frame_id

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=5):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)

    # Detect ball-and-bat collision
    collision_frame_id = detect_collision(cap)
    #print(f"Detected collision at frame {collision_frame_id}")

    # Extract frames starting from the collision frame
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, collision_frame_id)  # Start from collision frame

    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Resize frame to 224x224
        frame = cv2.resize(frame, (224, 224))

        # Quantize pixel values to integers (0-255)
        frame = np.uint8(frame)

        # Keep the frame as grayscale (1 channel)
        frame = np.expand_dims(frame, axis=-1)  # Shape: (224, 224, 1)

        frames.append(frame)

    cap.release()
    os.remove(video_path)  # Remove temp file
    return np.array(frames) if len(frames) == frame_count else None

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

# Prepare training data
batch_size = 30
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

# Create temp directory for videos
os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []

    # Process videos with proper tqdm progress tracking
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])

    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X)  # Shape: (batch_size, 5, 224, 224, 1)
    y = np.array(y)  # Shape: (batch_size, 3)

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')

    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    # Clear the TensorFlow session to release memory
    K.clear_session()

    # Trigger garbage collection to free up unused memory
    gc.collect()

    # Clear variables (X and y data) that are no longer needed
    X = []
    y = []

<ipython-input-2-74d2357d0100>:26: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=replicated_input, pooling='avg')
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed_1 (TimeDistributed) │ (None, 5, 1280)             │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │       1,573,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,964,995 (15.13 MB)

 Trainable params: 1,707,011 (6.51 MB)

...

Processing Videos: 100%|██████████| 30/30 [00:17<00:00,  1.75it/s]

Split data into train and test
Train model
Epoch 1/10


3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 2464.4436 - mae: 25.8492 - val_loss: 1434.3341 - val_mae: 21.7198
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 2808.0586 - mae: 27.1151 - val_loss: 1372.1949 - val_mae: 21.3111
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 1857.8444 - mae: 25.7752 - val_loss: 1231.9730 - val_mae: 20.2302
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 3540.2783 - mae: 28.4844 - val_loss: 1082.5768 - val_mae: 19.4149
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 1910.8398 - mae: 23.5083 - val_loss: 907.9352 - val_mae: 18.2710
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 3356.8706 - mae: 26.5617 - val_loss: 777.8880 - val_mae: 17.3100
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 2371.5488 - mae: 23.1532 - val_loss: 652.3516 - val_mae: 16.1891
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 1607.2435 - mae: 19.7073 - val_loss: 548.9636 - val_mae: 15.0407
Epoch 9/10
3/3 ━━━━━━━━━━━

Processing Videos:  50%|█████     | 15/30 [00:09<00:09,  1.62it/s]


KeyboardInterrupt: 

In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed, Input
from tensorflow.keras.optimizers import Adam
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import tensorflow.keras.backend as K
import gc

# Load pre-trained CNN model (feature extractor)
def create_grayscale_mobilenet():
    input_tensor = Input(shape=(128, 128, 1))  # Updated resolution to 128x128
    replicated_input = tf.keras.layers.Concatenate()([input_tensor] * 3)

    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=replicated_input, pooling='avg')
    base_model.trainable = False  

    feature_extractor = Model(inputs=input_tensor, outputs=base_model.output)
    return feature_extractor

feature_extractor = create_grayscale_mobilenet()

# Define the LSTM-based regression model
model = Sequential([
    TimeDistributed(feature_extractor, input_shape=(5, 128, 128, 1)),  # Updated resolution
    LSTM(256, return_sequences=False),  
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(3)  
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
model.summary()

# Function to download video
def download_video(video_url, save_path):
    response = requests.get(video_url, stream=True)
    with open(save_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    return save_path

# Function to detect ball-and-bat collision
def detect_collision(cap):
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  
    prev_frame = None
    collision_frame_id = 0

    for frame_id in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

        if prev_frame is None:
            prev_frame = gray_frame
            continue

        frame_diff = cv2.absdiff(prev_frame, gray_frame)
        _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if any(cv2.contourArea(contour) > 500 for contour in contours):
            collision_frame_id = frame_id
            break

        prev_frame = gray_frame

    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  
    return collision_frame_id

# Function to extract frames from video
def preprocess_video(video_url, save_path, frame_count=5):
    video_path = download_video(video_url, f'{save_path}.mp4')
    cap = cv2.VideoCapture(video_path)

    collision_frame_id = detect_collision(cap)

    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.set(cv2.CAP_PROP_POS_FRAMES, collision_frame_id)  

    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, (128, 128))  # Updated resolution

        frame = np.uint8(frame)  

        frame = np.expand_dims(frame, axis=-1)  

        frames.append(frame)

    cap.release()
    os.remove(video_path)  
    return np.array(frames, dtype=np.float16) if len(frames) == frame_count else None  # Use float16 to reduce size

# Load and merge datasets
df1 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2016-mlb-homeruns.csv')
df2 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2017-mlb-homeruns.csv')
df3 = pd.read_csv('/kaggle/input/mlb-fan-content-interaction/2024-mlb-homeruns.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)

batch_size = 16  # Reduced batch size for memory efficiency
num_batches = len(df) // batch_size + (len(df) % batch_size != 0)

os.makedirs("temp_videos", exist_ok=True)

for batch_idx in range(num_batches):
    print(f"Processing batch {batch_idx + 1}/{num_batches}")
    df_batch = df[batch_idx * batch_size:(batch_idx + 1) * batch_size]
    X, y = [], []

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(preprocess_video, row['video'], 'temp_videos/' + row['play_id']): row for _, row in df_batch.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Videos"):
            res = future.result()
            if res is not None:
                X.append(res)
                y.append([futures[future]['ExitVelocity'], futures[future]['HitDistance'], futures[future]['LaunchAngle']])

    if len(X) == 0:
        print("Skipping batch due to no valid videos.")
        continue

    X = np.array(X, dtype=np.float16)  
    y = np.array(y, dtype=np.float16)  

    print("Split data into train and test")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train model")
    model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

    print("Deleting Previous Saved model")
    if os.path.exists(f'model_batch_{batch_idx}.h5'):
        os.remove(f'model_batch_{batch_idx}.h5')

    print("Save model after each batch")
    model.save(f'model_batch_{batch_idx + 1}.h5')
    print(f"Model saved after batch {batch_idx + 1}")

    print("Evaluate model")
    loss, mae = model.evaluate(X_test, y_test)
    print(f'Batch {batch_idx + 1} - Test Loss: {loss}, Test MAE: {mae}')

    K.clear_session()
    gc.collect()

    X = []
    y = []


<ipython-input-1-166f508ce789>:22: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=replicated_input, pooling='avg')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 5, 1280)             │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 256)                 │       1,573,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

...

Split data into train and test
Train model
Epoch 1/10


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 380.9196 - mae: 13.1678 - val_loss: 147.4866 - val_mae: 7.2330
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 242.4663 - mae: 12.0816 - val_loss: 155.7843 - val_mae: 7.3532
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 186.3758 - mae: 10.1463 - val_loss: 164.3732 - val_mae: 7.4423
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 291.3875 - mae: 11.0587 - val_loss: 172.3923 - val_mae: 7.4455
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 485.0028 - mae: 14.1417 - val_loss: 179.1740 - val_mae: 7.4651
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 419.6431 - mae: 13.2852 - val_loss: 181.4986 - val_mae: 7.6942
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 255.8004 - mae: 11.9265 - val_loss: 182.8488 - val_mae: 7.9357
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 364.9121 - mae: 14.2854 - val_loss: 182.3547 - val_mae: 7.9517
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step

Processing Videos: 100%|██████████| 16/16 [00:09<00:00,  1.60it/s]

Split data into train and test
Train model
Epoch 1/10


KeyboardInterrupt: 